Использование Макспулов со страйдом 2 помогает бороться с проблемой пространственной локализации и маштабирования, при этом сильно снизить число параметров модели. По-этому, размер ядра по горизонтали можно не увиличивать ( увеличение размера ядра сильно приведет к увеличению параметров модели, можно , конечно, использовать pointwise и depthwise свертки, но мы ограничемся VGG16, так как эксперименты из предыдщущих статей указывают на то, что она работает довольно хорошо с данным датасетом). Модель довольно сильная, чтобы снизить шанс переобучения и выучивать более стабильные закономерности, было добавлено 2 DropOut после первого и второго ленейных слоев при p=0.4.

In [ ]:
class ConvBlock3(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.block= nn.Sequential(
        nn.Conv2d(in_channels= in_channels, out_channels= out_channels, kernel_size= 3, stride=1, padding= 1),
        #nn.BatchNorm2d(num_features= out_channels),
        nn.ReLU(inplace= True),

        nn.Conv2d(in_channels= out_channels, out_channels= out_channels, kernel_size= 3, stride= 1, padding= 1),
        #nn.BatchNorm2d(num_features= out_channels),
        nn.ReLU(inplace= True),

        nn.Conv2d(in_channels= out_channels, out_channels= out_channels, kernel_size= 3, stride= 1, padding= 1),
        #nn.BatchNorm2d(num_features= out_channels),
        nn.ReLU(inplace= True),

        nn.MaxPool2d(kernel_size=2, dilation= 1,  stride= 2)
    )

  def forward(self, x):
    return self.block(x)

In [ ]:
class ClassifyBlock(nn.Module):
  def __init__(self, in_features):
    super().__init__()

    self.block= nn.Sequential(
        nn.Linear(in_features= in_features, out_features= 4096),
        nn.ReLU(inplace= True),
        nn.Dropout(p= 0.5),

        nn.Linear(in_features= 4096, out_features= 4096),
        nn.ReLU(inplace= True),
        nn.Dropout(p= 0.5),

        nn.Linear(in_features= 4096, out_features= 50),
        nn.Softmax(dim= -1),
    )

  def forward(self, x):
    return self.block(x)

In [ ]:
class Model(nn.Module):
  def __init__(self, adappol_params= (2, 17)):
    super().__init__()
    self.in_block= nn.Sequential(
        nn.Conv2d(in_channels= 1, out_channels= 64, kernel_size= 3, stride=1, padding= 1),
        #nn.BatchNorm2d(num_features= 64 ),
        nn.ReLU(inplace= True),

        nn.Conv2d(in_channels= 64, out_channels= 64, kernel_size= 3, stride=1, padding= 1),
        #nn.BatchNorm2d(num_features= 64),
        nn.ReLU(inplace= True),

        nn.MaxPool2d(kernel_size=2, dilation=1, stride= 2),

        nn.Conv2d(in_channels= 64, out_channels= 128, kernel_size= 3, stride=1, padding= 1),
        #nn.BatchNorm2d(num_features= 128),
        nn.ReLU(inplace= True),

        nn.Conv2d(in_channels= 128, out_channels= 128, kernel_size= 3, stride=1, padding= 1),
        #nn.BatchNorm2d(num_features= 128),
        nn.ReLU(inplace= True),

        nn.MaxPool2d(kernel_size=2, dilation=1,  stride= 2)
    )


    self.conv_block= nn.Sequential(
        ConvBlock3(in_channels= 128, out_channels= 256),
        ConvBlock3(in_channels= 256, out_channels= 512),
        ConvBlock3(in_channels= 512, out_channels= 512)
    )

    self.flatt_block= nn.Sequential(
        nn.AdaptiveAvgPool2d(adappol_params),
        nn.Flatten()
    )

    in_features= 128*2**2*adappol_params[0]*adappol_params[1]
    self.classify_block= ClassifyBlock(in_features= in_features) # adappol_params*128*2**2



  def forward(self, x):
    x= self.in_block(x)
    x= self.conv_block(x)
    x= self.flatt_block(x)
    x= self.classify_block(x)
    return x
